In [7]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
bank_marketing = pd.read_csv('bank_marketing.csv')
client_cols = ['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']
campaign_cols = ['client_id', 'campaign', 'month', 'day', 'duration', 'pdays', 'previous', 'poutcome', 'y']
economics_cols = ['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']
client = bank_marketing[client_cols]
campaign = bank_marketing[campaign_cols]
economics = bank_marketing[economics_cols]


In [ ]:
client_cols_rn = {'client_id':'id'}
campaign_col_rn = {'duration':'contact_duration', 'previous':'previous_campaign_contacts', 'y':'campaign_outcome', 'poutcome':'previous_outcome', 'campaign':'number_contacts'}
economics_cols_rn = {'euribor3m':'euribor_three_months', 'nr_employed':'number_employed'}
client.rename(columns=client_cols_rn, inplace=True)
campaign.rename(columns=campaign_col_rn, inplace=True)
economics.rename(columns=economics_cols_rn, inplace=True)

In [ ]:
client['education'] = client['education'].str.replace('.', '_').replace('unknown',np.nan)

In [ ]:
client['job'] = client['job'].str.replace('.', '')

In [ ]:
campaign['previous_outcome'] = campaign['previous_outcome'].map({'success':1, 'failure':0})
campaign['previous_outcome'] = campaign['previous_outcome'].replace('nonexistent',np.nan)
campaign['campaign_outcome'] = campaign['campaign_outcome'].map({'success':1, 'failure':0})

In [8]:
campaign['campaign_id'] = "1"

In [26]:
campaign['last_contact_date'] = "2022" + "-" + campaign['month'].str.capitalize() + "-" + campaign['day'].astype(str)
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], format = "%Y-%b-%d")

In [39]:
client.to_csv('client.csv')
campaign.to_csv('campaign.csv')
economics.to_csv('economics.csv')

In [ ]:
marketing = pd.read_csv("bank_marketing.csv")

client = marketing[["client_id", "age", "job", "marital", "education", 
             "credit_default", "housing", "loan"]]
campaign = marketing[["client_id", "campaign", "month", "day", 
               "duration", "pdays", "previous", "poutcome", "y"]]
economics = marketing[["client_id", "emp_var_rate", "cons_price_idx", 
                "euribor3m", "nr_employed"]]

client.rename(columns={"client_id": "id"}, inplace=True)

campaign.rename(columns={"duration": "contact_duration", 
                         "y": "campaign_outcome", 
                         "campaign": "number_contacts",
                         "previous": "previous_campaign_contacts",
                         "poutcome": "previous_outcome"}, 
                         inplace=True)

economics.rename(columns={"euribor3m": "euribor_three_months", 
                          "nr_employed": "number_employed"}, 
                          inplace=True)

client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

client["job"] = client["job"].str.replace(".", "")

campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, 
                                                                 "no": 0})

campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", 
                                                                    np.NaN)
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, 
                                                                 "failure": 0})

campaign["campaign_id"] = 1

campaign["month"] = campaign["month"].str.capitalize()

campaign["year"] = "2022"

campaign["day"] = campaign["day"].astype(str)

campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%d")

campaign.drop(columns=["month", "day", "year"], inplace=True)

client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""